In [14]:
import os
import pandas as pd
import urllib
import datetime
import time

In [7]:
time.strftime("%d%m%Y")

'10052016'

In [8]:
start = time.time()
print "hello Jalal Jahir what"
print "hello Jalal Jahir is a GREAT Guy"
end = time.time()
processtime = end - start
print "{0: .3}".format(processtime)
print float(processtime)

hello Jalal Jahir what
hello Jalal Jahir is a GREAT Guy
 0.0
0.0


In [10]:
print os.getcwd()

J:\Staff\Jalal\Monthly_Emp_Update_Scratch\wa_esd_emp


In [11]:
current_dir = os.getcwd()

In [25]:
esd_sa_url = "https://fortress.wa.gov/esd/employmentdata/docs/economic-reports/wa-historical-employment-seasonally-adjusted.xlsx"
sa_filename = esd_sa_url[-49:-5]+"_"+time.strftime("%m%d%Y")+esd_sa_url[-5:]
def download_esd():
    start = time.time()
    esdfile = urllib.URLopener()
    esdfile.retrieve(esd_sa_url, os.getcwd()+"/"+ sa_filename)
    end = time.time()
    processtime = end - start
    print "{0: .3}".format(processtime)
    

In [26]:
download_esd()

 1.2


In [29]:
help (urllib)

Help on module urllib:

NAME
    urllib - Open an arbitrary URL.

FILE
    c:\anaconda2\lib\urllib.py

DESCRIPTION
    See the following document for more info on URLs:
    "Names and Addresses, URIs, URLs, URNs, URCs", at
    http://www.w3.org/pub/WWW/Addressing/Overview.html
    
    See also the HTTP spec (from which the error codes are derived):
    "HTTP - Hypertext Transfer Protocol", at
    http://www.w3.org/pub/WWW/Protocols/
    
    Related standards and specs:
    - RFC1808: the "relative URL" spec. (authoritative status)
    - RFC1738 - the "URL standard". (authoritative status)
    - RFC1630 - the "URI spec". (informational status)
    
    The object returned by URLopener().open(file) will differ per
    protocol.  All you know is that is has methods read(), readline(),
    readlines(), fileno(), close() and info().  The read*(), fileno()
    and close() methods work like those of open files.
    The info() method returns a mimetools.Message object which can be
    used t

In [385]:
df_king = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])
df_sa_king = pd.DataFrame(df_king)

In [386]:
df_sa_king.head()

,NAICS CELL,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,1990-09-01 00:00:00,...,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00
NAICS INDUSTRY,,,,,,,,,,,,,,,,,,,,,
Total Nonfarm,0,1096300,1098400,1101600,1103100,1106100,1111200,1113900,1119000,1115500,...,1591500,1590500,1594000,1594100,1594600,1598900,1602600,1607700,1615600,1619900
Total Private,5000000,951400,952200,956300,957600,960200,964400,966700,967500,967100,...,1376800,1382600,1392400,1385300,1380200,1382000,1390300,1394000,1400200,1402600
Goods Producing,6000000,277800,276800,278100,277300,277400,277900,278800,278100,277500,...,258600,261300,260900,260600,254500,252900,254100,251900,252700,254200
Mining and Logging,10000000,1900,1900,1900,2000,2000,2000,2000,2000,2000,...,800,800,800,800,700,700,700,700,800,700
Construction,20000000,61900,61800,62600,63200,63700,63800,63200,62900,62600,...,86400,87700,87600,87300,85000,83800,84800,85000,86700,88000


In [378]:
## df_sa_king["id2"] = df_sa_king.index

In [388]:
#df_sa_king.index

Index([u'Total Nonfarm', u'    Total Private', u'        Goods Producing',
       u'                Mining and Logging', u'                Construction',
       u'                    Construction of Buildings',
       u'                    Heavy and Civil Engineering Construction',
       u'                    Specialty Trade Contractors',
       u'                Manufacturing',
       u'                    Durable Goods Mfg',
       u'                        Fabricated Metal Product Mfg',
       u'                        Computer and Electronic Product Mfg',
       u'                            Navigational, Measuring, Electromedical and Control',
       u'                        Transportation Equipment Mfg',
       u'                            Aerospace Product and Parts Mfg',
       u'                    Nondurable Goods Mfg',
       u'                        Food Manufacturing',
       u'        Service Providing',
       u'            Trade, Transportation, and Utilities',
    

In [390]:
df_sa_king = df_sa_king.transpose()

In [391]:
df_sa_king.head()

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Other Services,Repair and Maintenance,Personal and Laundry Services,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services
NAICS CELL,0,5000000,6000000,10000000,20000000,20236000,20237000,20238000,30000000,31000000,...,80000000,80811000,80812000,80813000,90000000,90910000,90920000,90921611,90930000,90931611
1990-01-01 00:00:00,1096300,951400,277800,1900,61900,18400,8300,35200,214000,179600,...,40100,11400,11600,17100,144900,21700,42300,29800,80900,36400
1990-02-01 00:00:00,1098400,952200,276800,1900,61800,18500,8200,35100,213100,178700,...,40200,11500,11600,17100,146200,21700,42500,29800,82000,36500
1990-03-01 00:00:00,1101600,956300,278100,1900,62600,18900,8300,35400,213600,179200,...,40400,11600,11700,17100,145300,22000,42700,29900,80600,36500
1990-04-01 00:00:00,1103100,957600,277300,2000,63200,19300,8500,35400,212100,177400,...,40200,11700,11600,16900,145500,22100,42900,30100,80500,36600


In [392]:
df_sa_king['date_stamp']= df_sa_king.index

In [394]:
df_sa_king.head()

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Repair and Maintenance,Personal and Laundry Services,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp
NAICS CELL,0,5000000,6000000,10000000,20000000,20236000,20237000,20238000,30000000,31000000,...,80811000,80812000,80813000,90000000,90910000,90920000,90921611,90930000,90931611,NAICS CELL
1990-01-01 00:00:00,1096300,951400,277800,1900,61900,18400,8300,35200,214000,179600,...,11400,11600,17100,144900,21700,42300,29800,80900,36400,1990-01-01 00:00:00
1990-02-01 00:00:00,1098400,952200,276800,1900,61800,18500,8200,35100,213100,178700,...,11500,11600,17100,146200,21700,42500,29800,82000,36500,1990-02-01 00:00:00
1990-03-01 00:00:00,1101600,956300,278100,1900,62600,18900,8300,35400,213600,179200,...,11600,11700,17100,145300,22000,42700,29900,80600,36500,1990-03-01 00:00:00
1990-04-01 00:00:00,1103100,957600,277300,2000,63200,19300,8500,35400,212100,177400,...,11700,11600,16900,145500,22100,42900,30100,80500,36600,1990-04-01 00:00:00


In [397]:
df_sa_king= df_sa_king.ix[1:]

In [398]:
df_sa_king.head()

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Repair and Maintenance,Personal and Laundry Services,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp
1990-01-01 00:00:00,1096300,951400,277800,1900,61900,18400,8300,35200,214000,179600,...,11400,11600,17100,144900,21700,42300,29800,80900,36400,1990-01-01 00:00:00
1990-02-01 00:00:00,1098400,952200,276800,1900,61800,18500,8200,35100,213100,178700,...,11500,11600,17100,146200,21700,42500,29800,82000,36500,1990-02-01 00:00:00
1990-03-01 00:00:00,1101600,956300,278100,1900,62600,18900,8300,35400,213600,179200,...,11600,11700,17100,145300,22000,42700,29900,80600,36500,1990-03-01 00:00:00
1990-04-01 00:00:00,1103100,957600,277300,2000,63200,19300,8500,35400,212100,177400,...,11700,11600,16900,145500,22100,42900,30100,80500,36600,1990-04-01 00:00:00
1990-05-01 00:00:00,1106100,960200,277400,2000,63700,19500,8400,35800,211700,176900,...,11800,11600,16900,145900,22800,43200,30200,79900,36800,1990-05-01 00:00:00


In [399]:
df_sa_king2 = df_sa_king.reset_index(drop =True)

In [457]:
df_sa_king2

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp,year,datestamp
0,1096300,951400,277800,1900,61900,18400,8300,35200,214000,179600,...,17100,144900,21700,42300,29800,80900,36400,1990-01-01 00:00:00,1990,1990
1,1098400,952200,276800,1900,61800,18500,8200,35100,213100,178700,...,17100,146200,21700,42500,29800,82000,36500,1990-02-01 00:00:00,1990,1990
2,1101600,956300,278100,1900,62600,18900,8300,35400,213600,179200,...,17100,145300,22000,42700,29900,80600,36500,1990-03-01 00:00:00,1990,1990
3,1103100,957600,277300,2000,63200,19300,8500,35400,212100,177400,...,16900,145500,22100,42900,30100,80500,36600,1990-04-01 00:00:00,1990,1990
4,1106100,960200,277400,2000,63700,19500,8400,35800,211700,176900,...,16900,145900,22800,43200,30200,79900,36800,1990-05-01 00:00:00,1990,1990
5,1111200,964400,277900,2000,63800,19600,8400,35800,212100,177000,...,17100,146800,22000,43200,30200,81600,36900,1990-06-01 00:00:00,1990,1990
6,1113900,966700,278800,2000,63200,19200,8500,35500,213600,178200,...,17000,147200,21900,43800,30700,81500,35600,1990-07-01 00:00:00,1990,1990
7,1119000,967500,278100,2000,62900,19100,8400,35400,213200,177900,...,17100,151500,21500,43900,30700,86100,39400,1990-08-01 00:00:00,1990,1990
8,1115500,967100,277500,2000,62600,19000,8400,35200,212900,177300,...,17100,148400,21400,44100,30700,82900,37600,1990-09-01 00:00:00,1990,1990
9,1112200,963700,274800,2000,61400,18700,8200,34500,211400,175800,...,17100,148500,21200,44000,30600,83300,38000,1990-10-01 00:00:00,1990,1990


In [536]:
df_sa_king2["year"] = df_sa_king2["date_stamp"].dt.year

In [537]:
df_sa_king2.tail()

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp,year,datestamp
310,1598900,1382000,252900,700,83800,21900,7800,54100,168400,141200,...,23700,216900,21600,68300,51500,127000,56700,2015-11-01 00:00:00,2015,1990
311,1602600,1390300,254100,700,84800,22200,7400,55200,168600,142200,...,23200,212300,21400,64700,47700,126200,58200,2015-12-01 00:00:00,2015,1990
312,1607700,1394000,251900,700,85000,22000,6900,56100,166200,139800,...,25300,213700,21800,66400,49000,125500,58300,2016-01-01 00:00:00,2016,1990
313,1615600,1400200,252700,800,86700,22200,7000,57500,165200,138500,...,26700,215400,21900,67900,50600,125600,58500,2016-02-01 00:00:00,2016,1990
314,1619900,1402600,254200,700,88000,22100,7200,58700,165500,138200,...,26700,217300,21900,68800,51300,126600,58200,2016-03-01 00:00:00,2016,1990


In [526]:
get_year()

['0     1990-01-01\n1     1990-02-01\n2     1990-03-01\n3     1990-04-01\n4     1990-05-01\n5     1990-06-01\n6     1990-07-01\n7     1990-08-01\n8     1990-09-01\n9     1990-10-01\n10    1990-11-01\n11    1990-12-01\n12    1991-01-01\n13    1991-02-01\n14    1991-03-01\n15    1991-04-01\n16    1991-05-01\n17    1991-06-01\n18    1991-07-01\n19    1991-08-01\n20    1991-09-01\n21    1991-10-01\n22    1991-11-01\n23    1991-12-01\n24    1992-01-01\n25    1992-02-01\n26    1992-03-01\n27    1992-04-01\n28    1992-05-01\n29    1992-06-01\n         ...    \n285   2013-10-01\n286   2013-11-01\n287   2013-12-01\n288   2014-01-01\n289   2014-02-01\n290   2014-03-01\n291   2014-04-01\n292   2014-05-01\n293   2014-06-01\n294   2014-07-01\n295   2014-08-01\n296   2014-09-01\n297   2014-10-01\n298   2014-11-01\n299   2014-12-01\n300   2015-01-01\n301   2015-02-01\n302   2015-03-01\n303   2015-04-01\n304   2015-05-01\n305   2015-06-01\n306   2015-07-01\n307   2015-08-01\n308   2015-09-01\n309   20

In [525]:
def get_year():
    for y in df_sa_king2["date_stamp"]:
        year= df_sa_king2['date_stamp']
            print str(year)
            #print [str(year)[6:10]] 
    
    
    #year= str(df_sa_king['date_stamp'])[:4]
       # df_sa_king2["year"] = year
        
        #print  year
        
        #year = ""

IndentationError: unexpected indent (<ipython-input-525-32f6bd9fd78e>, line 4)

In [447]:
df_sa_king2

NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp,year,datestamp
0,1096300,951400,277800,1900,61900,18400,8300,35200,214000,179600,...,17100,144900,21700,42300,29800,80900,36400,1990-01-01 00:00:00,1990,1990
1,1098400,952200,276800,1900,61800,18500,8200,35100,213100,178700,...,17100,146200,21700,42500,29800,82000,36500,1990-02-01 00:00:00,1990,1990
2,1101600,956300,278100,1900,62600,18900,8300,35400,213600,179200,...,17100,145300,22000,42700,29900,80600,36500,1990-03-01 00:00:00,1990,1990
3,1103100,957600,277300,2000,63200,19300,8500,35400,212100,177400,...,16900,145500,22100,42900,30100,80500,36600,1990-04-01 00:00:00,1990,1990
4,1106100,960200,277400,2000,63700,19500,8400,35800,211700,176900,...,16900,145900,22800,43200,30200,79900,36800,1990-05-01 00:00:00,1990,1990
5,1111200,964400,277900,2000,63800,19600,8400,35800,212100,177000,...,17100,146800,22000,43200,30200,81600,36900,1990-06-01 00:00:00,1990,1990
6,1113900,966700,278800,2000,63200,19200,8500,35500,213600,178200,...,17000,147200,21900,43800,30700,81500,35600,1990-07-01 00:00:00,1990,1990
7,1119000,967500,278100,2000,62900,19100,8400,35400,213200,177900,...,17100,151500,21500,43900,30700,86100,39400,1990-08-01 00:00:00,1990,1990
8,1115500,967100,277500,2000,62600,19000,8400,35200,212900,177300,...,17100,148400,21400,44100,30700,82900,37600,1990-09-01 00:00:00,1990,1990
9,1112200,963700,274800,2000,61400,18700,8200,34500,211400,175800,...,17100,148500,21200,44000,30600,83300,38000,1990-10-01 00:00:00,1990,1990


In [528]:
df_sa_king2.loc[0]["date_stamp"].month

1

In [418]:
str(df_sa_king2.loc[0]["date_stamp"])[:4]

'1990'

In [531]:
df_sa_king2["date_stamp"].year

AttributeError: 'Series' object has no attribute 'year'

In [367]:
df_sa_king.head()

NAICS CELL,index,0,5000000,6000000,10000000,20000000,20236000,20237000,20238000,30000000,...,80000000,80811000,80812000,80813000,90000000,90910000,90920000,90921611,90930000,90931611
0,NAICS INDUSTRY,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineerin...,Specialty Trade Contractors,Manufacturing,...,Other Services,Repair and Maintenance,Personal and Laundry Services,Membership Associations and Or...,Government,Federal Government,Total State Government,State Government Education...,Total Local Government,Local Government Education...
1,1990-01-01 00:00:00,1096300,951400,277800,1900,61900,18400,8300,35200,214000,...,40100,11400,11600,17100,144900,21700,42300,29800,80900,36400
2,1990-02-01 00:00:00,1098400,952200,276800,1900,61800,18500,8200,35100,213100,...,40200,11500,11600,17100,146200,21700,42500,29800,82000,36500
3,1990-03-01 00:00:00,1101600,956300,278100,1900,62600,18900,8300,35400,213600,...,40400,11600,11700,17100,145300,22000,42700,29900,80600,36500
4,1990-04-01 00:00:00,1103100,957600,277300,2000,63200,19300,8500,35400,212100,...,40200,11700,11600,16900,145500,22100,42900,30100,80500,36600


In [368]:
df_sa_king["Total Nonfarm"].head()

KeyError: 'Total Nonfarm'

In [ ]:
df_sa_king.head()

In [247]:
# df_sa_king.drop(df_sa_king.columns[[1]], axis = 1).head()

In [256]:
df_sa_king= df_sa_king.drop(df_sa_king.columns[1:], axis = 1)
df_sa_king.head()

NAICS INDUSTRY,index
0,1990-01-01
1,1990-02-01
2,1990-03-01
3,1990-04-01
4,1990-05-01


In [249]:
df_sa_king= df_sa_king.reset_index()

In [251]:
df_sa_king.head()

NAICS INDUSTRY,index,Total Nonfarm
0,1990-01-01,1096300
1,1990-02-01,1098400
2,1990-03-01,1101600
3,1990-04-01,1103100
4,1990-05-01,1106100


In [252]:
df_sa_king.index

RangeIndex(start=0, stop=315, step=1)

In [253]:
df_sa_king.columns

Index([u'index', u'Total Nonfarm'], dtype='object', name=u'NAICS INDUSTRY')

In [132]:
xl_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)

In [133]:
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [177]:
dfs["Yakima MSA"].head()

,Yakima MSA Quarterly Benchmarked,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316
0,NAICS CELL,NAICS INDUSTRY,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,...,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00
1,00000000,Total Nonfarm,63600,63600,64100,64300,64500,65000,65000,65000,...,82700,82400,84300,83900,84400,84200,84900,84900,85000,85300
2,05000000,Total Private,51700,51700,52100,52200,52300,52600,52400,52600,...,65600,64800,65600,65600,66200,66100,66500,66900,67200,67600
3,06000000,Goods Producing,12200,12100,12500,12400,12600,12800,12800,12800,...,12400,12200,12700,12900,12900,12500,12200,12200,12100,12300
4,15000000,"Mining, Logging, and Construction",2400,2300,2400,2400,2500,2500,2500,2600,...,3800,3800,3800,3800,3800,3800,3700,3400,3400,3600
